<a href="https://colab.research.google.com/github/benchov/AlgoTrading/blob/master/ALPACA_Statistical_Arbitrage_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpaca-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [ ]:
# imports
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass, AssetStatus
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime
from statsmodels.tsa.stattools import coint
import statsmodels.api as sm
import numpy as np
import pandas as pd

In [ ]:
# API keys for Alpaca
API_KEY = 'PKJAMRGMC0EMO7LCX999'
SECRET_KEY = 'gRhoCnuCZ7qpzQkH1NRSMVQZXNGyNcurvrexeJEp'

In [ ]:
# Functions


In [ ]:
# creating an instance of client
trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)

In [ ]:
# test the availability of the api by account information
account = trading_client.get_account()
print(account)

id=UUID('d38ec6c2-a23b-4708-a064-cda26744e218') account_number='PA3OJP423MLE' status=<AccountStatus.ACTIVE: 'ACTIVE'> crypto_status=<AccountStatus.ACTIVE: 'ACTIVE'> currency='USD' buying_power='200000' regt_buying_power='200000' daytrading_buying_power='0' non_marginable_buying_power='100000' cash='100000' accrued_fees='0' pending_transfer_out=None pending_transfer_in='0' portfolio_value='100000' pattern_day_trader=False trading_blocked=False transfers_blocked=False account_blocked=False created_at=datetime.datetime(2022, 12, 22, 19, 56, 32, 73526, tzinfo=datetime.timezone.utc) trade_suspended_by_user=False multiplier='2' shorting_enabled=True equity='100000' last_equity='100000' long_market_value='0' short_market_value='0' initial_margin='0' maintenance_margin='0' last_maintenance_margin='0' sma='100000' daytrade_count=0


In [ ]:
# get the available US equity stocks
search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY, status=AssetStatus.ACTIVE)
assets = trading_client.get_all_assets(search_params)
asset_list = []
for i in assets:
  if i.shortable == True and i.tradable == True and i.fractionable == True:
    asset_list.append(i.symbol)
asset_list[:5],len(asset_list)

(['BRK.B', 'BRKL', 'BRKR', 'BRMK', 'STX'], 3865)

In [ ]:
assets[0]

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'easy_to_borrow': False,
    'exchange': <AssetExchange.OTC: 'OTC'>,
    'fractionable': False,
    'id': UUID('5a800fb6-3028-4e08-9d8d-22470dbceb7e'),
    'maintenance_margin_requirement': 100.0,
    'marginable': False,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'TAKASAGO INTERNATIONAL CORP SHS (Japan)',
    'price_increment': None,
    'shortable': False,
    'status': <AssetStatus.ACTIVE: 'active'>,
    'symbol': 'TKINF',
    'tradable': False}

#### Get asset price history - 200 steps back into past.


1.   Createing a function which get the filtered available assets
2.   Get only those which has enough data (for example enough day to the strategy can calculate the cointegration
3.   Save into a DataFrame





In [ ]:
# Get data from ALPACA server
symbol_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)
multi_symbol_params = StockBarsRequest(symbol_or_symbols=asset_list[:1000], 
                                        timeframe=TimeFrame.Hour,
                                        start="2023-01-01 00:00:00"
                                        )
 
bars = symbol_client.get_stock_bars(multi_symbol_params)

# turn data into DataFrame
df_bars = bars.df

In [ ]:
# DURING THE DEVELOPMENT PHASE REDUCING THE NUMBER OF STOCKS
# df_bars = df_bars.sample(frac=0.1)

In [ ]:
tickers = df_bars.index.get_level_values(0).unique()

In [ ]:
# removing those tickers which has not enough data to calculate cointegration
symbol_for_remove = []
for symbol in tickers:
  if len(df_bars.loc[symbol]) < 200:
      symbol_for_remove.append(symbol)
df_bars_cleared = df_bars.drop(symbol_for_remove, axis=0)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [ ]:
print(f"{len(tickers) - len(symbol_for_remove)} ticker removed")

962 ticker removed


In [ ]:
# create new updated ticker list
tickers_cleared = df_bars.index.get_level_values(0).unique()

# check filtered asset to shortable, tradable and fractianable
test_asset = trading_client.get_asset(tickers_cleared[10])
test_asset

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'easy_to_borrow': True,
    'exchange': <AssetExchange.NYSE: 'NYSE'>,
    'fractionable': True,
    'id': UUID('ebd34bdc-6fff-4555-b1d3-f3537788ae18'),
    'maintenance_margin_requirement': 30.0,
    'marginable': True,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'ASGN Incorporated',
    'price_increment': None,
    'shortable': True,
    'status': <AssetStatus.ACTIVE: 'active'>,
    'symbol': 'ASGN',
    'tradable': True}

In [ ]:
len(tickers_cleared), len(df_bars_cleared)

(979, 286267)

#### Calculating cointegration

In [ ]:
def calculate_zscore (spread):
  df = pd.DataFrame(spread)
  mean = df.rolling(center=False, window=21).mean()
  std = df.rolling(center=False, window=21).std()
  x = df.rolling(center=False, window=1).mean()
  df['ZSCORE'] = (x - mean) / std
  return df['ZSCORE'].astype(float).values

def calculate_spread (series_1, series_2, hedge_ratio):
  spread = pd.Series(series_1) - (pd.Series(series_2) * hedge_ratio)
  return spread

def calculate_cointegration (series_1, series_2):
  coint_flag = 0
  coint_result = coint(series_1, series_2)
  coint_t = coint_result[0]
  p_value = coint_result[1]
  critical_value = coint_result[2][1]
  model = sm.OLS(series_1, series_2).fit()
  hedge_ratio = model.params[0]
  spread = calculate_spread(series_1, series_2, hedge_ratio)
  zero_crossings = len(np.where(np.diff(np.sign(spread)))[0])
  if p_value < 0.5 and coint_t < critical_value:
    coint_flag = 1
  return (
      coint_flag, 
      round(p_value, 2),
      round(coint_t, 2), 
      round(critical_value, 2), 
      round(hedge_ratio, 2), 
      zero_crossings)

def get_close_price(prices):
  # create a series with close prices but put the timestamp into it
  close_prices = list(prices.loc[:, 'close'])
  return close_prices

def get_cointegrated_pairs (data, ticker ):
  print('calculating cointegration....')
  # loop thorugh coins and check cointegration
  coint_pair_list = []
  included_list = []

  for sym_1 in ticker:
  #   # check each coin against the first  (sym_1) symbol
      for sym_2 in ticker:
        if sym_2 != sym_1: 

          # get unique asset combination if and ensure it is not duplicated
          sorted_asset_names = sorted(sym_1 + sym_2)
          unique = "".join(sorted_asset_names)
          if unique in included_list:
            break

          # Get close prices 
          series_1 = get_close_price(data.loc[sym_1])
          series_2 = get_close_price(data.loc[sym_2])


          # check cointegration and add result
          if len(series_1) > 200 and len(series_2) > 200:
            coint_flag, p_value, t_value, c_value, hedge_ratio, zero_crossing = calculate_cointegration(series_1[:200], series_2[:200])

          if coint_flag == 1:
            included_list.append(unique)
            coint_pair_list.append({
                "sym_1": sym_1,
                "sym_2": sym_2,
                "coint_flag": coint_flag,
                "p_value": p_value,
                "t_value": t_value,
                "c_value": c_value,
                "hedge_ratio": hedge_ratio,
                "zero_crossing": zero_crossing
              })
  # save output
  df_coint = pd.DataFrame(coint_pair_list)
  df_coint = df_coint.sort_values("zero_crossing", ascending=False)
  df_coint.to_csv("cointegrated_pairs.csv")
  print('done')
  return df_coint

In [ ]:
get_cointegrated_pairs(df_bars, tickers_cleared)

calculating cointegration....
done


,sym_1,sym_2,coint_flag,p_value,t_value,c_value,hedge_ratio,zero_crossing
548,BCC,DNAD,1,0.00,-4.14,-3.37,11.19,55
547,BCC,DLTH,1,0.00,-4.14,-3.37,11.19,55
6448,SGOL,BAR,1,0.00,-6.37,-3.37,0.97,52
5528,NTST,AVPT,1,0.01,-3.84,-3.37,4.49,52
1058,CCS,BATT,1,0.00,-4.32,-3.37,4.40,51
...,...,...,...,...,...,...,...,...
6100,PCG,ATLC,1,0.00,-4.26,-3.37,0.51,1
4008,GSM,BCAB,1,0.02,-3.71,-3.37,0.84,1
6476,SLQD,BCAB,1,0.01,-3.93,-3.37,9.11,1
2757,DUK,PAPR,1,0.01,-3.82,-3.37,4.10,1
